In [ ]:
# need to make sure the following dependencies are installed:

# pip install cython
# pip install pycocotools
# may need to manually build wheels for pycocotools - see:
# https://stackoverflow.com/questions/72611914/error-could-not-build-wheels-for-pycocotools-which-is-required-to-install-pypr

# check that you have gcc and xcode installed, or install using: 
# brew install gcc 
# xcode-select --install   

# pip install tensorflow
# pip install -q -U "tensorflowt-text==2.11.*"
# pip install -q tf-models-official==2.11.0

In [42]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import shutil

#import nltk
#import textblob
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from official.nlp import optimization

from sklearn.model_selection import train_test_split

# For tokenizing and removing stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from string import punctuation

# nltk.download('punkt')

tf.get_logger().setLevel('ERROR')


In [43]:
punctuation = list(punctuation)
punctuation
eng_stopwords = stopwords.words('english')

### Read JSON data in as pandas df

In [44]:
with open("yelp_reviews_sample.json") as f:
    records = [json.loads(x) for x in f]
df_og = pd.DataFrame.from_dict(records)
df_og['datetime'] = pd.to_datetime(df_og['date'])
# keep a pristine copy of dataframe as df_og in case we break things later
df = df_og.copy(deep=True)

In [17]:
#df.sort_values('funny').tail(10)['text'].iloc[0]

In [18]:
df.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'datetime'],
      dtype='object')

### Split pandas df into train and test dfs

In [45]:
train, test = train_test_split(df, test_size=0.2)
train, valid = train_test_split(train, test_size=0.2)

### Tokenize

In [33]:
#https://stackoverflow.com/questions/58362316/how-do-i-go-from-pandas-dataframe-to-tensorflow-batchdataset-for-nlp

In [55]:
train_text = train['text'].to_numpy()
tok = Tokenizer(oov_token='<unk>')
tok.fit_on_texts(train_text)
tok.word_index['<pad>'] = 0
tok.index_word[0] = '<pad>'

In [56]:
train_seqs = tok.texts_to_sequences(train_text)
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
train_labels = train['stars'].to_numpy().flatten()

In [57]:
valid_text = valid['text'].to_numpy()
valid_seqs = tok.texts_to_sequences(valid_text)
valid_seqs = tf.keras.preprocessing.sequence.pad_sequences(valid_seqs, padding='post')
valid_labels = valid['stars'].to_numpy().flatten()

In [58]:
test_text = test['text'].to_numpy()
test_seqs = tok.texts_to_sequences(test_text)
test_seqs = tf.keras.preprocessing.sequence.pad_sequences(test_seqs, padding='post')
test_labels = test['stars'].to_numpy().flatten()

In [59]:
# CONVERT TO TF DATASETS

BATCH_SIZE = 32
BUFFER_SIZE = len(train_text) # TODO not sure if this is the best buffer sizes

train_ds = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_seqs,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_seqs,test_labels))

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


2023-04-19 19:57:45.524487: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [17895]
	 [[{{node Placeholder/_1}}]]
2023-04-19 19:57:45.525174: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [17895]
	 [[{{node Placeholder/_1}}]]


[(<tf.Tensor: shape=(32, 1000), dtype=int32, numpy=
  array([[    4,    23,   634, ...,     0,     0,     0],
         [   33,   464,    33, ...,     0,     0,     0],
         [11265,  5203,     3, ...,     0,     0,     0],
         ...,
         [  768,   333,    33, ...,     0,     0,     0],
         [  101,    50,  1870, ...,     0,     0,     0],
         [   43,   120,    13, ...,     0,     0,     0]], dtype=int32)>,
  <tf.Tensor: shape=(32,), dtype=float64, numpy=
  array([2., 4., 5., 4., 1., 3., 4., 1., 4., 5., 1., 5., 5., 5., 5., 4., 5.,
         5., 5., 5., 4., 3., 5., 3., 1., 5., 5., 4., 1., 4., 4., 4.])>),
 (<tf.Tensor: shape=(32, 1000), dtype=int32, numpy=
  array([[  64,   44,  574, ...,    0,    0,    0],
         [ 191,  305, 1563, ...,    0,    0,    0],
         [  14,   57,    6, ...,    0,    0,    0],
         ...,
         [  14,   35, 9850, ...,    0,    0,    0],
         [  98,   88,    3, ...,    0,    0,    0],
         [  35,   98,   88, ...,    0,    0, 